In [1]:
import numpy as np
import pandas as pd
import keras
import utilities_LR
from pathlib import Path
from sklearn import metrics

In [2]:
config_n5 = {
     'data': {
        'n_datasets': 45_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 5, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        #'n_informative': random.randint(2, 10),
        'n_informative': 'random',
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        ### int or 'random'
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 1,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 42,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False, # should be always false
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae',# keras.losses.BinaryCrossentropy(from_logits=False), #tf.keras.losses.get(config['lambda_net']['loss_lambda']), # 'mae'
            'metrics': ['mae', keras.losses.BinaryCrossentropy(from_logits=False)]
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 500,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True, # usually true
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False,
                'stratify': None
            },
            'train_noise': 0 # y_flip fraction on Y_train pred data from lambda net
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 256,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 49,
        'use_gpu': True,
        'gpu_numbers': '1',
        'RANDOM_SEED': 1,   
    }
}

In [3]:
config_n5_noise = {
     'data': {
        'n_datasets': 45_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 5, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        #'n_informative': random.randint(2, 10),
        'n_informative': 'random',
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        ### int or 'random'
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 1,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 42,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False, # should be always false
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae',# keras.losses.BinaryCrossentropy(from_logits=False), #tf.keras.losses.get(config['lambda_net']['loss_lambda']), # 'mae'
            'metrics': ['mae', keras.losses.BinaryCrossentropy(from_logits=False)]
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 500,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True, # usually true
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False,
                'stratify': None
            },
            'train_noise': 0.1 # y_flip fraction on Y_train pred data from lambda net
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 256,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 49,
        'use_gpu': True,
        'gpu_numbers': '1',
        'RANDOM_SEED': 1,   
    }
}

In [4]:
config_n10 = {
     'data': {
        'n_datasets': 45_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 10, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        #'n_informative': random.randint(2, 10),
        'n_informative': 'random',
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        ### int or 'random'
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 1,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 44,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False, # should be always false
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae',# keras.losses.BinaryCrossentropy(from_logits=False), #tf.keras.losses.get(config['lambda_net']['loss_lambda']), # 'mae'
            'metrics': ['mae', keras.losses.BinaryCrossentropy(from_logits=False)]
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 500,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True, # usually true
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False,
                'stratify': None
            },
            'train_noise': 0 # y_flip fraction on Y_train pred data from lambda net
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 256,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 49,
        'use_gpu': True,
        'gpu_numbers': '1',
        'RANDOM_SEED': 1,   
    }
}

In [5]:
config_n10_noise = {
     'data': {
        'n_datasets': 45_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 10, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        #'n_informative': random.randint(2, 10),
        'n_informative': 'random',
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        ### int or 'random'
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 1,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 44,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False, # should be always false
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae',# keras.losses.BinaryCrossentropy(from_logits=False), #tf.keras.losses.get(config['lambda_net']['loss_lambda']), # 'mae'
            'metrics': ['mae', keras.losses.BinaryCrossentropy(from_logits=False)]
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 500,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True, # usually true
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False,
                'stratify': None
            },
            'train_noise': 0.1 # y_flip fraction on Y_train pred data from lambda net
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 256,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 49,
        'use_gpu': True,
        'gpu_numbers': '1',
        'RANDOM_SEED': 1,   
    }
}

In [6]:
config_n20 = {
     'data': {
        'n_datasets': 45_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 20, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        #'n_informative': random.randint(2, 10),
        'n_informative': 'random',
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        ### int or 'random'
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 1,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 46,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False, # should be always false
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae',# keras.losses.BinaryCrossentropy(from_logits=False), #tf.keras.losses.get(config['lambda_net']['loss_lambda']), # 'mae'
            'metrics': ['mae', keras.losses.BinaryCrossentropy(from_logits=False)]
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 500,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True, # usually true
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False,
                'stratify': None
            },
            'train_noise': 0 # y_flip fraction on Y_train pred data from lambda net
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 256,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 49,
        'use_gpu': True,
        'gpu_numbers': '1',
        'RANDOM_SEED': 1,   
    }
}

In [7]:
config_n20_noise = {
     'data': {
        'n_datasets': 45_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 20, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        #'n_informative': random.randint(2, 10),
        'n_informative': 'random',
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        ### int or 'random'
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 1,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 46,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False, # should be always false
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae',# keras.losses.BinaryCrossentropy(from_logits=False), #tf.keras.losses.get(config['lambda_net']['loss_lambda']), # 'mae'
            'metrics': ['mae', keras.losses.BinaryCrossentropy(from_logits=False)]
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 500,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True, # usually true
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False,
                'stratify': None
            },
            'train_noise': 0.1 # y_flip fraction on Y_train pred data from lambda net
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 256,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 49,
        'use_gpu': True,
        'gpu_numbers': '1',
        'RANDOM_SEED': 1,   
    }
}

# Inets DT

In [8]:
config_DT_n5 = {
    'data': {
        'n_features': 5,
        'noise': 0
    }
}

In [9]:
config_DT_n5_noise = {
    'data': {
        'n_features': 5,
        'noise': 0.1
    }
}

In [10]:
config_DT_n10 = {
    'data': {
        'n_features': 10,
        'noise': 0
    }
}

In [11]:
config_DT_n10_noise = {
    'data': {
        'n_features': 10,
        'noise': 0.1
    }
}

In [12]:
config_DT_n20 = {
    'data': {
        'n_features': 20,
        'noise': 0
    }
}

In [13]:
config_DT_n20_noise = {
    'data': {
        'n_features': 20,
        'noise': 0.1
    }
}

In [14]:
def load_res_DT(config):
    path = "./data"
    
    #return pd.read_csv(directory + '/evalRes_plain_logreg.csv'), pd.read_csv(directory + '/evalRes_valid.csv')
    inetDT_evalRes = pd.read_csv(path + f"/evalRes_inetDT_n{config['data']['n_features']}_noise{config['data']['noise']}.csv")
    return inetDT_evalRes


In [15]:
inetDT_n5           = load_res_DT(config_DT_n5       )
inetDT_n5_noise     = load_res_DT(config_DT_n5_noise )
inetDT_n10          = load_res_DT(config_DT_n10      )
inetDT_n10_noise    = load_res_DT(config_DT_n10_noise)
inetDT_n20          = load_res_DT(config_DT_n20)
inetDT_n20_noise    = load_res_DT(config_DT_n20_noise)

In [16]:
print(inetDT_n5       .shape)
print(inetDT_n5_noise .shape)
print(inetDT_n10      .shape)
print(inetDT_n10_noise.shape)
print(inetDT_n20_noise.shape)
print(inetDT_n20_noise.shape)

(1001, 12)
(1001, 12)
(1001, 12)
(1001, 12)
(1001, 12)
(1001, 12)


In [17]:
def load_res(config):
    path = utilities_LR.inet_path_LR(config)
    
    #return pd.read_csv(directory + '/evalRes_plain_logreg.csv'), pd.read_csv(directory + '/evalRes_valid.csv')
    plain_logreg = pd.read_csv(path + f"/evalRes_plain_logreg_n{config['data']['n_features']}_noise{config['inets']['data_prep']['train_noise']}.csv")
    plain_DT     = pd.read_csv(path + f"/evalRes_plain_DT_n{config['data']['n_features']}_noise{config['inets']['data_prep']['train_noise']}.csv")
    valid        = pd.read_csv(path + f"/evalRes_valid_n{config['data']['n_features']}_noise{config['inets']['data_prep']['train_noise']}.csv")
    return plain_DT, plain_logreg, valid


In [18]:
plain_DT_n5       , plain_logreg_n5       , valid_n5        = load_res(config_n5       )
plain_DT_n5_noise , plain_logreg_n5_noise , valid_n5_noise  = load_res(config_n5_noise )
plain_DT_n10      , plain_logreg_n10      , valid_n10       = load_res(config_n10      )
plain_DT_n10_noise, plain_logreg_n10_noise, valid_n10_noise = load_res(config_n10_noise)
plain_DT_n20      , plain_logreg_n20      , valid_n20       = load_res(config_n20      )
plain_DT_n20_noise, plain_logreg_n20_noise, valid_n20_noise = load_res(config_n20_noise)

In [19]:
plain_DT_n5

,Unnamed: 0,index_0=aggregated,mse,tn,fp,fn,tp,precision,recall,f1,ROC-AUC,MCC
0,0,0.0,0.062675,351.869333,23.011111,23.995111,351.124444,0.940010,0.935653,0.936939,0.936917,0.875788
1,1,0.0,0.038667,380.000000,14.000000,15.000000,341.000000,0.960563,0.957865,0.959212,0.961166,0.922460
2,2,1.0,0.130667,347.000000,41.000000,57.000000,305.000000,0.881503,0.842541,0.861582,0.868436,0.738640
3,3,2.0,0.056000,365.000000,34.000000,8.000000,343.000000,0.909814,0.977208,0.942308,0.945997,0.890179
4,4,3.0,0.073333,326.000000,41.000000,14.000000,369.000000,0.900000,0.963446,0.930643,0.925865,0.855269
...,...,...,...,...,...,...,...,...,...,...,...,...
4496,4496,4495.0,0.042667,344.000000,16.000000,16.000000,374.000000,0.958974,0.958974,0.958974,0.957265,0.914530
4497,4497,4496.0,0.030667,346.000000,17.000000,6.000000,381.000000,0.957286,0.984496,0.970701,0.968832,0.938952
4498,4498,4497.0,0.177333,309.000000,43.000000,90.000000,308.000000,0.877493,0.773869,0.822430,0.825855,0.651820
4499,4499,4498.0,0.014667,361.000000,6.000000,5.000000,378.000000,0.984375,0.986945,0.985658,0.985298,0.970655


In [20]:
def get_Acc(df):
    accvec = (df.tp+df.tn)/(df.tn + df.fp + df.fn + df.tp)
    return accvec.mean()

In [21]:
print('inet_logreg_n5        ', get_Acc(valid_n5       ))
print('inet_logreg_n5_noise  ', get_Acc(valid_n5_noise ))
print('inet_logreg_n10       ', get_Acc(valid_n10      ))
print('inet_logreg_n10_noise ', get_Acc(valid_n10_noise))
print('inet_logreg_n20       ', get_Acc(valid_n20      ))
print('inet_logreg_n20_noise ', get_Acc(valid_n20_noise))
print()
print('plain_logreg_n5       ', get_Acc(plain_logreg_n5       ))
print('plain_logreg_n5_noise ', get_Acc(plain_logreg_n5_noise ))
print('plain_logreg_n10      ', get_Acc(plain_logreg_n10      ))
print('plain_logreg_n10_noise', get_Acc(plain_logreg_n10_noise))
print('plain_logreg_n20      ', get_Acc(plain_logreg_n20      ))
print('plain_logreg_n20_noise', get_Acc(plain_logreg_n20_noise))
print()
print('inet_DT_n5            ', get_Acc(inetDT_n5       ))
print('inet_DT_n5_noise      ', get_Acc(inetDT_n5_noise ))
print('inet_DT_n10           ', get_Acc(inetDT_n10      ))
print('inet_DT_n10_noise     ', get_Acc(inetDT_n10_noise))
print('inet_DT_n20           ', get_Acc(inetDT_n20      ))
print('inet_DT_n20_noise     ', get_Acc(inetDT_n20_noise))
print()
print('plain_DT_n5           ', get_Acc(plain_DT_n5       ))
print('plain_DT_n5_noise     ', get_Acc(plain_DT_n5_noise ))
print('plain_DT_n10          ', get_Acc(plain_DT_n10      ))
print('plain_DT_n10_noise    ', get_Acc(plain_DT_n10_noise))
print('plain_DT_n20          ', get_Acc(plain_DT_n20      ))
print('plain_DT_n20_noise    ', get_Acc(plain_DT_n20_noise))

inet_logreg_n5         0.9124765777777777
inet_logreg_n5_noise   0.9130072444444445
inet_logreg_n10        0.8211678666666666
inet_logreg_n10_noise  0.8230250666666666
inet_logreg_n20        0.500878311111111
inet_logreg_n20_noise  0.5285556888888888

plain_logreg_n5        0.955104888888889
plain_logreg_n5_noise  0.8644112592592593
plain_logreg_n10       0.9461389629629631
plain_logreg_n10_noise 0.8571075555555556
plain_logreg_n20       0.9481860740740741
plain_logreg_n20_noise 0.8586453333333333

inet_DT_n5             0.464082
inet_DT_n5_noise       0.440782
inet_DT_n10            0.549214
inet_DT_n10_noise      0.478942
inet_DT_n20            0.47949200000000003
inet_DT_n20_noise      0.464418

plain_DT_n5            0.9373250370370372
plain_DT_n5_noise      0.8500296296296296
plain_DT_n10           0.9171751111111112
plain_DT_n10_noise     0.8340562962962962
plain_DT_n20           0.8705801481481481
plain_DT_n20_noise     0.796234962962963
